# [GD14]Video_app_kalman

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4


In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출  ,ㅂㅏㄹㄱㄱㅣ 50% down
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 50/403 [00:00<00:00, 493.09it/s, now=None]

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult2.mp4


In [4]:
# CASE 1 : moviepy 사용 read/write time
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 48/403 [00:00<00:00, 384.00it/s, now=None]

Moviepy - Building video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.76ms


In [5]:
# CASE 2 : OpenCV 사용  // read/write time
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.53ms


# 루브릭

- moviepy 를 이용할 때의 장단점을 분석해 봅시다. 주피터 노트북에 답변을 작성해 코드와 함께 제출해 주세요.
 * MoviePy 특징
   + 간단하며 직관적임
   + Flexible함
   + Protable함
   + numpy와의 호환성
   + 단점으로는 stream video에 대한 작업엔 적합하지 않습니다. 또 비디오의 개수가 많을 경우(100개 이상) 적합하지 않습니다.


1. 웹캠을 통한 스티커앱을 실행하고 다각도로 문제점을 분석하여 보았다.
 - 거리, 인원수, 각도 등 다양한 측면에서의 문제점과 기술적 대안을 정리하였다.
2. 스티커앱 초기버전이 가진 예외사항들을 분석하여 수정하였다. 
 - 프레임 좌표범위 예외처리 관련 오류를 수정하였다.
3. 칼만 필터를 웹캠 스티커앱에 적용하여 스티커의 안정성을 높여 보았다.
 - 칼만 필터를 적용한 스티커앱을 작성하여 실행해 보고 안정성 여부를 확인하였다.


1. 첫번째 사진 
 - 거리, 각도, 프레임 벗어나면 잘 안됨
 - https://drive.google.com/file/d/1Htnx-FLIXjtbZApCNQDAj4wXRCQWjEIE/view?usp=sharing
<img width="80%" src="https://user-images.githubusercontent.com/38655888/117322260-ce22a000-aec8-11eb-942e-8855cb86fbbf.gif"/>
2. 두번째 사진 
 - 거리, 각도, 프레임은 해결된듯하나 속도가 4배 느려짐
 - https://drive.google.com/file/d/1RvGALyKkgK1Qn9agmXt3DLsD9X14WyHh/view?usp=sharing
<img width="80%" src="https://user-images.githubusercontent.com/38655888/117322535-0de98780-aec9-11eb-8482-32f502cf1ec2.gif"/>
3. 세번째 사진 
 - 칼만필터 적용
 - https://drive.google.com/file/d/1mx5F8y0HaCGr4QCmr9qkMSTyAEa9-9T4/view?usp=sharing
<img width="80%" src="https://user-images.githubusercontent.com/38655888/117322680-2c4f8300-aec9-11eb-818c-dcb4884b134a.gif"/>

In [ ]:
## webcam_sticker.py 내용
## from kfaddsticker import img2sticker_kf 추가하여 img2sticker_kf 함수 사용

import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker
from kfaddsticker import img2sticker_kf

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    #vc = cv2.VideoCapture(0)
    #vc = cv2.VideoCapture('rtmp://rtmp.streamaxia.com/streamaxia/c23CC6')   # XXXXXX 부분은 본인 어플리케이션에서 확인한 코드로 대체해 주세요. 
    vc = cv2.VideoCapture('./images/video2.mp4')
    
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        #img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   
        img_result = img2sticker_kf(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()


In [ ]:
## newaddsticker.py 내용
## preprocess, detector, landmark 변경

import dlib
import cv2

def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    # detector
    dlib_rects = detector_hog(img_rgb, 0)
    if len(dlib_rects) < 1:
        return img_orig
    
    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
    
    # head coord
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[30][0] # nose
        y = landmark[30][1] - dlib_rect.width()//2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break
    
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)

    refined_x = x - w // 2
    refined_y = y - h

    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    ###
    # TODO : x 축 예외처리 코드 추가
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
    ###

    img_bgr = img_orig.copy()
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr




